In [1]:
from main import init_config
from controller import Controller
from dataset.process import TokenParser

import numpy as np
import tensorflow as tf

#To use all gpu-s
ctrl = Controller(*init_config())
strategy = tf.distribute.MirroredStrategy()
BATCH_SIZE_2 = 4*len(tf.config.list_physical_devices('GPU'))
print( f"Inference will run on {len(tf.config.list_physical_devices('GPU'))} gpus.")

2022-01-22 21:43:30.174289: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Running the DreamProlog algorithm.
There was a problem with the provided arguments. The program will run in the default setting:
--configs prolog --logdir logdir
Namespace(act='elu', action_gnn_hidden_act=64, action_gnn_hidden_val=200, action_gnn_layers=6, action_gnn_next_shape=(32, 64, 32), action_gnn_start_shape=(4, 1, 4), action_repeat=1, actor_disc=5, actor_dist='normal', actor_entropy='1e-4', actor_grad_clip=100, actor_init_std=1.0, actor_layers=4, actor_lr=0.001, actor_min_std=0.1, actor_outscale=0.0, actor_state_entropy=0.0, actor_temp=0.1, atari_grayscale=False, behavior_stop_grad=True, clip_rewards='identity', collect_dyn_sample=True, dataset_size=100000, debug=False, disag_layers=1, disag_log=True, disag_models=3, disag_offset=0, disag_target='embed', disag_units=200, discount=0.95, discount_lambda=0.8, discount_layers=2, discount_scale=1.0, dyn_cell='gru', dyn_deter=512, dyn_discrete=32, dyn_hidden=512, dyn_input_layers=5, dyn_mean_act='none', dyn_min_std=0.1, dyn_output_lay

usage: ipykernel_launcher.py [-h] --configs CONFIGS [CONFIGS ...] --logdir
                             LOGDIR
ipykernel_launcher.py: error: the following arguments are required: --configs, --logdir


Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t17_subset_1.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Suite: prolog
Found 131 problem files.
Query:
    init_python("leancop/theorems/m2n140/t40_tex_2.p",[conj, nodef, eager_reduction(1)],GnnInput, SimpleFeatures, TextFeatures, TextActions, ActionsMask, Result) 

Prefill dataset (0 steps).


/home/kristsz/DreamProLog/envs/wrappers.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  value = np.array(value)


Eval episode has 23 steps and return -0.200.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Inference will run on 2 gpus.


2022-01-22 21:43:55.843179: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-22 21:43:55.843832: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-22 21:43:57.114502: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-01-22 21:43:57.115152: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:09:00.0 name: TITAN V computeCapability: 7.0
coreClock: 1.455GHz coreCount: 80 deviceMemorySize: 11.78GiB deviceMemoryBandwidth: 607.97GiB/s
2022-01-22 21:43:57.115172: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-22 21:43:57.116846: I

In [2]:
#Parsing meta datas
meta = ctrl.datasetManager._train_eps._meta
ds1 = []
for v in meta.values():
    ds1.extend([str(s)[1:] for s in v['action_space_text']])
    
parser = TokenParser()

def pad(narr):
    size = narr.size
    if size>128:
        narr = narr[:128]
        size = 128
    return np.pad(narr, [1, 128-size], constant_values= [(299, 0)])


In [3]:
#episodes = ctrl.datasetManager._train_eps._storage
episodes = ctrl.datasetManager._train_eps._episodes
ds2 = []
#for pname, peps in episodes.items():
#    print(pname)
#    for length, eps in peps.items():
#        print(length, ':', len(eps), sep='\t')
     
for ep in episodes.values():
    for state in ep['text']:
        ds2.extend([str(s)[1:] for s in state])
        
ds = ds2 + ds1
parsed_ds = []
for dp in ds:
    parsed_ds.append(pad(np.array(parser.parse(dp), dtype = np.int32)))

In [4]:
#This is a dataset which feeds goal strings
import tensorflow as tf
BATCH_SIZE = 32
def generator():
    for i in range(1000):
        x = tf.constant([parsed_ds[np.random.randint(len(parsed_ds))] for i in range(BATCH_SIZE)])
        yield x[:, :-1], x[:, 1:]
sgn = tf.TensorSpec((BATCH_SIZE, 128), dtype=tf.int32)
tf_ds = tf.data.Dataset.from_generator(generator, output_signature = (sgn, sgn))

In [5]:
ds = iter(tf_ds)
next(ds)

2022-01-22 21:45:06.908389: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-22 21:45:06.925430: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3598190000 Hz


(<tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[299, 344,  23, ...,   0,   0,   0],
        [299,   9,   5, ...,   0,   0,   0],
        [299,   9,   5, ...,   0,   0,   0],
        ...,
        [299,   9,   5, ...,   0,   0,   0],
        [299,  36,  23, ...,   0,   0,   0],
        [299, 231,   0, ...,   0,   0,   0]], dtype=int32)>,
 <tf.Tensor: shape=(32, 128), dtype=int32, numpy=
 array([[344,  23,  15, ...,   0,   0,   0],
        [  9,   5, 184, ...,   0,   0,   0],
        [  9,   5, 241, ...,   0,   0,   0],
        ...,
        [  9,   5,  12, ...,   0,   0,   0],
        [ 36,  23,  12, ...,   0,   0,   0],
        [231,   0,   0, ...,   0,   0,   0]], dtype=int32)>)

In [6]:
storage = ctrl.datasetManager._train_eps._storage
def extract_ep_names(eps):
    ep_names = []
    ep_length = []
    for k, v in eps.items():
        ep_names.extend([name for _, name in v])
        ep_length.extend([stat[2] for stat, name in v])
    ep_length = np.array(ep_length, dtype = np.float32)
    ep_length/=np.sum(ep_length)
    return (ep_names, ep_length)
refactored_storage = {name: extract_ep_names(eps) for name, eps in storage.items()}

In [7]:
#BATCH_SIZE_2 = 4
problems = list(storage.keys())
def generator2():
    for i in range(100):
        problem = np.random.choice(problems)
        ep_names, probs = refactored_storage[problem]
        eps = np.random.choice(ep_names, size = (BATCH_SIZE_2,), p = probs)
        res = []
        for name in eps:
            ep = episodes[name]
            length = len(ep['action'])
            q = np.random.randint(length)
            state = ep['text'][q]
            state = np.array([pad(np.array(parser.parse(str(goal)[1:]), dtype = np.int32)) for goal in state])
            state = np.pad(state[:128], [(0, 128-state.shape[0]), (0,0)])
            res.append(state)
        res = np.stack(res)
        yield ((res[:, :, :-1], res[:, :, 1:]), tf.zeros((1,), tf.int32))
        
sgn = tf.TensorSpec((BATCH_SIZE_2, 128, 128), dtype = tf.int32)
tf_ds2 = tf.data.Dataset.from_generator(generator2, output_signature = ((sgn, sgn), tf.TensorSpec((1,), tf.int32)))

In [8]:
sample = next(iter(tf_ds2))

In [9]:
from unorganized.parts5 import Net, Encoder, Decoder

GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [10]:
class Loss(tf.keras.losses.Loss):
    def __init__(self, omega=0.0):
        super().__init__()
        self.omega = omega
        self.loss_objective = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        
    def call(self, y_true, y_pred):
        loss = self.loss_objective(y_true, y_pred)
        mask = 1-tf.cast(tf.math.equal(y_true, 0), tf.float32)
        
        loss *= mask
        return loss

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=6000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [11]:
class StateNet(tf.keras.Model):
    def __init__(self, ):
        super().__init__()
        num_layers = 3
        embed_tokens = 512
        querry = 16
        self.goal_querry = querry
        length = 128
        d_model = 128
        dff = 512
        num_heads = 8
        dropout_rate = 0.1
        d_scale = 4
        self.goal_embedder = Net(num_layers, embed_tokens, querry, length, d_model, num_heads, dff, dropout_rate)
        self.latent_dense = tf.keras.layers.Dense(d_model*d_scale)
        self.state_encoder = Encoder(N = 3, querry = 16, output_length = 128, 
                                     d_model = d_model*d_scale, num_heads = num_heads, dff = dff*d_scale, rate = dropout_rate)
        self.state_decoder = Decoder(N = 3, output_length = 128, 
                                     d_model = d_model*d_scale, num_heads = num_heads, dff = dff*d_scale, rate = dropout_rate)
        self.out_dense = tf.keras.layers.Dense(d_model*self.goal_querry)
        
        self._loss = Loss()
        self._loss2 = tf.keras.losses.MeanSquaredError()
        #lr = CustomSchedule(d_model, 1000)
        #self._opt = tf.optimizers.Adam(lr, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
        
    def encode(self, inp, mask, training):
        batch_size = inp.shape[0]
        x = tf.reshape(inp, (128*batch_size, 128))
        goal_embed, goal_mask = self.goal_embedder.encode(x, training)
        #goal_embed = tf.reshape(x, (batch_size, 128) + (8, 128))
        x = tf.reshape(goal_embed, (batch_size, 128, self.goal_querry * 128))
        x = self.latent_dense(x)
        
        y, _ = self.state_encoder(x, mask, training)
        return y, goal_embed, goal_mask
    
    def decode(self, x, mask, training):
        x = self.state_decoder(x, mask, training)
        x = self.out_dense(x)
        x = tf.reshape(x, x.shape[:-1]+(self.goal_querry, 128))
        return x
        
    
    def calc_loss(self, inp, target, training):
        mask = tf.cast(tf.math.equal(inp[:, :, 1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
        latent, goal_embed, goal_mask = self.encode(inp, mask, training)
        decoded_from_goals = self.goal_embedder.decode(goal_embed, goal_mask, training)
        decoded_from_goals = tf.reshape(decoded_from_goals, (4, 128,) + decoded_from_goals.shape[1:])
        goal_embed_reconst = self.decode(latent, mask, training)
        goal_embed_reconst = tf.reshape(goal_embed_reconst, (4*128,) + goal_embed_reconst.shape[2:])
        decoded_from_embed = self.goal_embedder.decode(goal_embed_reconst, goal_mask, training)
        decoded_from_embed = tf.reshape(decoded_from_embed, (4, 128,) + decoded_from_embed.shape[1:])
        
        losses = {}
        losses['goal_embedder'] = self._loss(target, decoded_from_goals)
        losses['goal_to_goal'] = self._loss(target, decoded_from_embed)
        losses['state_to_state'] = self._loss2(goal_embed, goal_embed_reconst)
        return losses
    
    def call(self, x, training):
        inp, target = x
        losses = self.calc_loss(inp, target, training)
        scalars = {
                'goal_embedder': 2.5,
                'goal_to_goal': 0,
                'state_to_state': 0,
            }
        loss = 0.0
        for k, l in losses.items():
            loss += scalars[k]*l
        return loss
    
    #@tf.function(input_signature=(sgn, sgn))
    def train_step(self, data):
        data, _ = data
        with tf.GradientTape() as tape:
            loss = self(data, True)
            
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        
               
        return {'loss': tf.reduce_sum(loss)}
               
    

In [12]:
model = StateNet()

In [13]:
from unorganized.embedder_state import StateNet
model = StateNet()
learning_rate = CustomSchedule(128, 100)
optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
model.compile(optimizer = optimizer)

In [14]:
#learning_rate = CustomSchedule(d_model)
#optimizer=tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)
#goal_embedder.compile(optimizer = optimizer, loss=Loss(), metrics=[])

In [15]:
#goal_embedder.fit(tf_ds, epochs = 30)

In [16]:
sample = next(iter(tf_ds2))
inp, target = sample[0]
model.calc_loss(inp, target, False)

2022-01-22 21:45:08.088668: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-22 21:45:08.896736: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


{'goal_embedder': <tf.Tensor: shape=(), dtype=float32, numpy=2480967000.0>,
 'goal_to_goal': <tf.Tensor: shape=(), dtype=float32, numpy=2475300000.0>,
 'state_to_state': <tf.Tensor: shape=(), dtype=float32, numpy=22377.707>}

In [17]:
model.fit(tf_ds2, epochs = 80)

Epoch 1/80
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (

2022-01-22 21:45:39.359259: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 64.00MiB (rounded to 67108864)requested by op state_net_1/decoder_2/deep_cross_attention_11/simple_layer_45/dense_274/Tensordot/MatMul
Current allocation summary follows.
2022-01-22 21:45:39.359402: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-01-22 21:45:39.359429: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 71, Chunks in use: 69. 17.8KiB allocated for chunks. 17.2KiB in use in bin. 904B client-requested in use in bin.
2022-01-22 21:45:39.359450: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 374, Chunks in use: 374. 187.5KiB allocated for chunks. 187.5KiB in use in bin. 187.0KiB client-requested in use in bin.
2022-01-22 21:45:39.359466: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 2, Chunks 

2022-01-22 21:45:39.479530: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f147db8db00 of size 67108864 next 1681
2022-01-22 21:45:39.479544: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f1481b8db00 of size 8388608 next 1684
2022-01-22 21:45:39.479552: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f148238db00 of size 2097152 next 1685
2022-01-22 21:45:39.479560: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f148258db00 of size 268435456 next 1686
2022-01-22 21:45:39.479568: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f149258db00 of size 33554432 next 1688
2022-01-22 21:45:39.479577: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f149458db00 of size 2097152 next 1689
2022-01-22 21:45:39.479585: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f149478db00 of size 8388608 next 1690
2022-01-22 21:45:39.479593: I tensorflow/core/common_runtime/bfc_allocat

2022-01-22 21:45:49.486091: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.00MiB (rounded to 1048576)requested by op state_net_1/decoder_3/deep_cross_attention_13/simple_layer_56/dense_337/Tensordot/MatMul
Current allocation summary follows.
2022-01-22 21:45:49.486236: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-01-22 21:45:49.486263: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 71, Chunks in use: 69. 17.8KiB allocated for chunks. 17.2KiB in use in bin. 904B client-requested in use in bin.
2022-01-22 21:45:49.486284: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 378, Chunks in use: 378. 189.5KiB allocated for chunks. 189.5KiB in use in bin. 189.0KiB client-requested in use in bin.
2022-01-22 21:45:49.486301: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 2, Chunks in

2022-01-22 21:45:49.631466: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f146b18db00 of size 2097152 next 1669
2022-01-22 21:45:49.631476: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f146b38db00 of size 2097152 next 1672
2022-01-22 21:45:49.631486: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f146b58db00 of size 65536 next 1682
2022-01-22 21:45:49.631496: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f146b59db00 of size 65536 next 1683
2022-01-22 21:45:49.631506: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f146b5adb00 of size 65536 next 1696
2022-01-22 21:45:49.631516: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f146b5bdb00 of size 65536 next 1697
2022-01-22 21:45:49.631526: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f146b5cdb00 of size 65536 next 1716
2022-01-22 21:45:49.631536: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] In

2022-01-22 21:45:59.635529: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 8.00MiB (rounded to 8388608)requested by op state_net_1/decoder_3/deep_cross_attention_13/simple_layer_55/dense_335/Tensordot/MatMul
Current allocation summary follows.
2022-01-22 21:45:59.635675: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-01-22 21:45:59.635702: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 71, Chunks in use: 69. 17.8KiB allocated for chunks. 17.2KiB in use in bin. 904B client-requested in use in bin.
2022-01-22 21:45:59.635723: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 378, Chunks in use: 378. 189.5KiB allocated for chunks. 189.5KiB in use in bin. 189.0KiB client-requested in use in bin.
2022-01-22 21:45:59.635739: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 2, Chunks in

2022-01-22 21:45:59.651199: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eaa2b00 of size 512 next 981
2022-01-22 21:45:59.651202: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eaa2d00 of size 512 next 982
2022-01-22 21:45:59.651206: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eaa2f00 of size 512 next 983
2022-01-22 21:45:59.651210: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eaa3100 of size 512 next 984
2022-01-22 21:45:59.651213: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eaa3300 of size 512 next 985
2022-01-22 21:45:59.651217: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eaa3500 of size 65536 next 986
2022-01-22 21:45:59.651220: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eab3500 of size 512 next 987
2022-01-22 21:45:59.651224: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f127eab3700 of 

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[131072,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node state_net_1/decoder_2/deep_cross_attention_11/simple_layer_45/dense_274/Tensordot/MatMul (defined at home/kristsz/DreamProLog/unorganized/attention.py:98) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[gradient_tape/state_net_1/embedding_1/embedding_lookup/Reshape/_32]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[131072,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node state_net_1/decoder_2/deep_cross_attention_11/simple_layer_45/dense_27

In [ ]:
sample = next(iter(tf_ds2))
inp, target = sample[0]
model.calc_loss(inp, target, False)

In [ ]:
inp, target = sample[0]
inp

In [ ]:
self = model
training = False
mask = tf.cast(tf.math.equal(inp[:, :, 1], 0), tf.float32)[:, tf.newaxis, :, tf.newaxis]
latent, goal_embed, goal_mask = self.encode(inp, mask, training)
decoded_from_goals = self.goal_embedder.decode(goal_embed, goal_mask, training)
decoded_from_goals = tf.reshape(decoded_from_goals, (4, 128,) + decoded_from_goals.shape[1:])
goal_embed_reconst = self.decode(latent, mask, training)
goal_embed_reconst = tf.reshape(goal_embed_reconst, (4*128,) + goal_embed_reconst.shape[2:])
decoded_from_embed = self.goal_embedder.decode(goal_embed_reconst, goal_mask, training)
decoded_from_embed = tf.reshape(decoded_from_embed, (4, 128,) + decoded_from_embed.shape[1:])
        

In [ ]:
goal_embed, goal_embed_reconst

In [ ]:
y_true = target
_y_pred = decoded_from_embed
_y_pred = tf.nn.softmax(_y_pred, axis=-1)
y_probs = tf.gather(_y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(_y_pred, axis=-1)
y_pred[:, :4, :8], y_true[:, :4, :8], tf.cast(100*y_probs[:, :4, :8], tf.int32)

In [ ]:
y_true = target
_y_pred = decoded_from_embed
_y_pred = tf.nn.softmax(_y_pred, axis=-1)
y_probs = tf.gather(_y_pred, y_true, axis=-1, batch_dims=2)
y_pred = tf.math.argmax(_y_pred, axis=-1)
y_pred[:, :4, :8], y_true[:, :4, :8], tf.cast(100*y_probs[:, :4, :8], tf.int32)

In [ ]:
latent-latent[:, :1, :]

In [ ]:
decoded_goal_embed = model.decode(latent, mask, False)
decoded_goal_embed

d78db00 of size 8388608 next 1723
2022-01-22 21:45:59.865314: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f14adf8db00 of size 8388608 next 1722
2022-01-22 21:45:59.865335: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f14ae78db00 of size 2097152 next 1732
2022-01-22 21:45:59.865354: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f14ae98db00 of size 2097152 next 1733
2022-01-22 21:45:59.865373: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f14aeb8db00 of size 2097152 next 1734
2022-01-22 21:45:59.865400: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f14aed8db00 of size 2097152 next 1725
2022-01-22 21:45:59.865419: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f14aef8db00 of size 2097152 next 1726
2022-01-22 21:45:59.865437: I tensorflow/core/common_runtime/bfc_allocator.cc:1028] InUse at 7f14af18db00 of size 8388608 next 1729
2022-01-22 21:45:59.865454: I tensorflow/c